In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

In [ ]:
d0 = 11
dd0 = 64

#geom,dpy = 'Thwaites_1.0',10
geom,dpy = 'Thwaites_0.5',2

forc,moviename = 'tanh_Tdeep0.5_ztcl-500',f'Thwaites/{geom}_cold_SSAfloat'
#forc,moviename = 'sose_R2',f'Thwaites/{geom}_warm_SSAfloat'

In [ ]:
ds0 = xr.open_dataset(f'../results/{geom}_{forc}_{d0:03.0f}.nc')

for d,d1 in enumerate(np.arange(d0,dd0+1)):

    fig,ax = plt.subplots(1,4,figsize=(12,5))

    for dax in ax:
        dax.set_xticks([])
        dax.set_yticks([])
        dax.set_aspect('equal','box')

    ds = xr.open_dataset(f'../results/{geom}_{forc}_{d1:03.0f}.nc')

    Nyears = (d1-d0)/dpy
    for dax in ax:
        dax.pcolormesh(ds.x,ds.y,ds.mask,cmap='cmo.ice',vmin=-5,vmax=3)
        
    im = ax[0].pcolormesh(ds.x,ds.y,np.where(ds.mask>0,ds.H,np.nan),vmin=0,vmax=1500,cmap='gist_stern')
    cb = plt.colorbar(im,ax=ax[0],orientation='horizontal')
    cb.set_label('Ice thickness [m]')

    im = ax[1].pcolormesh(ds.x,ds.y,np.where(ds.tmask==1,ds.melt,np.nan),norm=mpl.colors.LogNorm(vmin=1, vmax=100),cmap='inferno')
    cb = plt.colorbar(im,ax=ax[1],orientation='horizontal')
    cb.set_label('Melt [m/yr]')
    ax[1].set_title(f'Mean melt: {np.nanmean(np.where(ds.tmask==1,ds.melt,np.nan)):+2.1f} m/yr')

    vel = (ds.Ui**2+ds.Vi**2)**.5
    im = ax[2].pcolormesh(ds.x,ds.y,np.where(ds.mask>0,vel,np.nan),norm=mpl.colors.LogNorm(vmin=100, vmax=10000),cmap='cmo.amp')
    cb = plt.colorbar(im,ax=ax[2],orientation='horizontal')
    cb.set_label('Ice speed [m/yr]')
    ax[2].streamplot(ds.x.values,ds.y.values,np.where(ds.mask>0,ds.Ui,np.nan),ds.Vi,color='k',density=2,linewidth=.5,arrowsize=.1)
    #ax[2].set_title(f'Mean melt: {np.nanmean(np.where(ds.tmask==1,ds.melt,np.nan)):+2.1f} m/yr')

    maskdiff = np.maximum(-1,np.minimum(1,ds.mask-ds0.mask))
    im = ax[3].pcolormesh(ds.x,ds.y,maskdiff,vmin=-2,vmax=2,cmap='cmo.balance_r')
    cb = plt.colorbar(im,ax=ax[3],orientation='horizontal')
    cb.set_label('Calving / Advance')

    ds.close()

    ax[0].set_title(f'Year {Nyears: 2.1f}')

    plt.savefig(f'../videos/tmp/frame_{d:03.0f}')
    print(d,Nyears)
    plt.close()
ds0.close()
print('Finished')

Create movie. First need to delete old movie manually for security

In [ ]:
os.system(f'ffmpeg -r 4 -f image2 -i ../videos/tmp/frame_%03d.png -pix_fmt yuv420p ../videos/{moviename}.mp4')
os.system(f'rm -r ../videos/tmp/*')